In [1]:
import numpy as np
import sys
sys.path.append('/Users/eframe/dmi/src')
import eventAnalysis as ea
import tables
import matplotlib.pyplot as plt
from matplotlib import cm
import codedAperture as ca
import calibrate as calib
import scipy
from matplotlib.animation import FuncAnimation
import matplotlib
from mpl_toolkits import mplot3d
from matplotlib.font_manager import FontProperties

In [2]:
def animate(i):
    surf[0].remove()
    surf[0] = ax.plot_surface(X, Y, val[i, :, :], color='grey', alpha = 0.8, rstride=1, cstride=1, linewidth=0.8, edgecolors='k')
    return surf

In [3]:
def animate_rect(i):
    rect.set_xy(xy[i])
    rect.set_visible(True)
    return rect

In [4]:
mFile = '/Users/eframe/dmi/mask.mat'
data = scipy.io.loadmat(mFile)
mask = data['Mopt']

In [26]:
m = 1
n_m = len( mask.flatten() )
n_s = len( mask.flatten() ) / ( 4 * m ** 2 ) # total number of bins in submatrix
a = int( np.sqrt( n_s ) / 2 )
b = int( np.sqrt( n_m ) / 2 )
A = mask[ b-a:b+a, b-a:b+a ].flatten() # submatrix in the center of array at a specific magnifications
A = A / np.std( A )
shifts = int( np.sqrt( n_m ) - np.sqrt( n_s ) ) + 1
val = []
I = int( np.sqrt( n_s ) )
for i in np.arange( shifts ):
    for j in np.arange( shifts ):
        G = mask[ i:i + I, j:j + I ].flatten()
        G = G / np.std( G )
        psf = np.correlate(A, G)
        val.extend( psf )
val = np.array( val ).reshape( ( shifts, shifts ) ) / max(val) / 2

In [27]:
N = int( ( np.sqrt(n_m) - np.sqrt(n_s) + 1 ) ** 2 ) # total number of submatrices
N

1089

In [28]:
sidelobes = np.delete( val.flatten(), np.argmax(val) )
var = np.std( sidelobes )
side = np.mean( sidelobes )
max_side = np.max(sidelobes)
print(side, var,  max_side)

0.24991092654589173 0.007475076934852142 0.270060253722356


In [29]:
min(var.flatten())

0.007475076934852142

In [30]:
%matplotlib qt
font = FontProperties()
font.set_name('Times New Roman')
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.set_aspect('auto')
x, y = np.arange( val.shape[0] ), np.arange( val.shape[0] )
X, Y = np.meshgrid(x, y)
surf = [ax.plot_surface(X, Y, val[:, :], color='grey', alpha = 0.5, rstride=1, cstride=1, linewidth=0.8, edgecolors='k')]
ax.zaxis.set_rotate_label(False)
ax.set_zlabel('correlation', rotation = 270, fontsize=50,fontproperties=font, labelpad=50)
ax.set_xlabel('u shift', fontsize=50, fontproperties=font, labelpad=35)
ax.set_ylabel('v shift', fontsize=50, fontproperties=font, labelpad=35)
ax.tick_params( labelsize=50 )
ax.tick_params( axis='z',pad=20 )
ax.zaxis.set_ticks(np.arange(0.1, 0.6, 0.1))
for l in ax.xaxis.get_ticklabels():
    l.set_family("Times New Roman")
    l.set_size(50)
for l in ax.yaxis.get_ticklabels():
    l.set_family("Times New Roman")
    l.set_size(50)
for l in ax.zaxis.get_ticklabels():
    l.set_family("Times New Roman")
    l.set_size(50)

In [19]:
A1 = mask[32:, :32].flatten()
A2 = mask[16:48, 16:48].flatten()
A1 = A1 / A1.std()
A2 = A2 / A2.std()

val1, val2 = [], []
for i in np.arange(33):
    for j in np.arange(33):
        G = mask[i:i + 32, j:j + 32].flatten()
        G = G / G.std()
        psf = np.correlate(A1, G)
        val1.extend(psf)
for i in np.arange(33):
    for j in np.arange(33):
        G = mask[i:i + 32, j:j + 32].flatten()
        G = G / G.std()
        psf = np.correlate(A2, G)
        val2.extend(psf)
val1 = np.array(val1)/max(val1)/2
val2 = np.array(val2)/max(val2)/2
val = np.array([val1, val2])

fig = plt.figure()
xe, ye = np.arange(33), np.arange(33)
X, Y = np.meshgrid(xe, ye)
val = val.reshape(2, 33, 33)
ax = plt.axes(projection='3d')
surf = [ax.plot_surface(X, Y, val[0, :, :], color='grey', alpha = 0.8, rstride=1, cstride=1, linewidth=0.8, edgecolors='k')]
ax.set_zlabel('corr', fontsize=20, rotation=270, labelpad=15)
ax.tick_params(labelsize=15)

In [64]:
%matplotlib qt
fig, ax = plt.subplots()

xe, ye = np.arange( 17 ), np.arange( 17 )
X, Y = np.meshgrid( xe, ye )
im = ax.pcolormesh( X, Y, mask[:16, :16].T, cmap='Greys')
ax.set_xlabel('i', fontsize = 50, fontproperties=font)
ax.set_ylabel('j', rotation=360, fontsize = 50, fontproperties=font, labelpad=15)
ax.tick_params( labelsize=50 )
ax.xaxis.set_ticks(np.arange(0, 17, 4))
ax.yaxis.set_ticks(np.arange(0, 17, 4))
ax.set_aspect('equal', adjustable='box')
for l in ax.xaxis.get_ticklabels():
    l.set_family("Times New Roman")
    l.set_size(50)
for l in ax.yaxis.get_ticklabels():
    l.set_family("Times New Roman")
    l.set_size(50)